In [1]:
import transformers
print(transformers.__version__)

4.51.3


In [13]:
!pip uninstall -y transformers
!pip install transformers==4.53.0 --upgrade --no-cache-dir


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found existing installation: transformers 4.51.3
Uninstalling transformers-4.51.3:
  Successfully uninstalled transformers-4.51.3


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 177.5 MB/s eta 0:00:000:00:01


In [14]:
with open('/kaggle/input/sentimental/texts.txt', 'r', encoding='utf-8') as f:
    texts = [line.strip() for line in f]

In [15]:
with open('/kaggle/input/sentimental/test_texts.txt', 'r', encoding='utf-8') as f:
    test_texts = [line.strip() for line in f]

In [16]:
with open('/kaggle/input/sentimental/test_labels.txt', 'r', encoding='utf-8') as f:
    test_labels = [line.strip() for line in f]

In [17]:
with open('/kaggle/input/sentimental/labels.txt', 'r', encoding='utf-8') as f:
    labels = [line.strip() for line in f]

In [18]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize

In [19]:
x_train=texts
x_test=test_texts
y_train=labels
y_test=test_labels

In [20]:
from datasets import Dataset
import pandas as pd

train_df = pd.DataFrame({'text': x_train, 'label': y_train})
test_df = pd.DataFrame({'text': x_test, 'label': y_test})

#  Hugging Face Dataset format convo
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)


In [21]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length")

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [22]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch

# Number of classes (e.g., 2 for binary sentiment)
num_labels = len(set(y_train))

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",   # ✅ enables evaluation each epoch
    logging_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,   # optional, if you want best checkpoint
    logging_dir='./logs',          # optional: logs will be saved here
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [24]:
!pip install transformers torch

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.8 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.10.19
    Uninstalling nvidia-curand-cu12-

In [27]:
import torch
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
from transformers import pipeline
import pandas as pd
import numpy as np

RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
cannot import name 'auto_docstring' from 'transformers.utils' (/usr/local/lib/python3.11/dist-packages/transformers/utils/__init__.py)

In [26]:
# Uninstall and reinstall transformers
!pip uninstall transformers -y
!pip install transformers --upgrade

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found existing installation: transformers 4.53.0
Uninstalling transformers-4.53.0:
  Successfully uninstalled transformers-4.53.0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 86.2 MB/s eta 0:00:00:00:01:01


In [28]:
# Complete clean installation
!pip uninstall transformers tokenizers -y
!pip install --no-cache-dir transformers torch

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found existing installation: transformers 4.53.0
Uninstalling transformers-4.53.0:
  Successfully uninstalled transformers-4.53.0
Found existing installation: tokenizers 0.21.1
Uninstalling tokenizers-0.21.1:
  Successfully uninstalled tokenizers-0.21.1


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 194.3 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 361.2 MB/s eta 0:00:00


In [31]:
import torch
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
# Don't import pipeline for now
import pandas as pd
import numpy as np

# Use BERT directly without pipeline
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Example usage
text = "This is a test sentence"
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

with torch.no_grad():
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state
    
print("BERT loaded successfully!")
print(f"Embedding shape: {embeddings.shape}")

BERT loaded successfully!
Embedding shape: torch.Size([1, 7, 768])


In [33]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW  # Import AdamW from torch.optim instead
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
import torch.nn.functional as F

y_train_int = [int(label) for label in y_train]
y_test_int = [int(label) for label in y_test]

print(f"Training samples: {len(x_train)}")
print(f"Test samples: {len(x_test)}")
print(f"Unique labels in train: {set(y_train_int)}")

model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        # Tokenize the text
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


train_dataset = TextDataset(x_train, y_train_int, tokenizer)
test_dataset = TextDataset(x_test, y_test_int, tokenizer)


batch_size = 16  
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 3  

print(f"Using device: {device}")
print(f"Training batches: {len(train_loader)}")
print(f"Test batches: {len(test_loader)}")


def train_epoch(model, train_loader, optimizer, device):
    model.train()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0
    
    for batch in train_loader:

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
   
        optimizer.zero_grad()

        outputs = model(input_ids=input_ids, 
                       attention_mask=attention_mask, 
                       labels=labels)
        
        loss = outputs.loss
        logits = outputs.logits
        

        loss.backward()
        optimizer.step()
        

        predictions = torch.argmax(logits, dim=-1)
        correct_predictions += (predictions == labels).sum().item()
        total_predictions += labels.size(0)
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_loader)
    accuracy = correct_predictions / total_predictions
    
    return avg_loss, accuracy


def evaluate_model(model, test_loader, device):
    model.eval()
    total_loss = 0
    all_predictions = []
    all_labels = []
    
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model(input_ids=input_ids, 
                           attention_mask=attention_mask, 
                           labels=labels)
            
            loss = outputs.loss
            logits = outputs.logits
            
            predictions = torch.argmax(logits, dim=-1)
            
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            total_loss += loss.item()
    
    avg_loss = total_loss / len(test_loader)
    accuracy = accuracy_score(all_labels, all_predictions)
    
    return avg_loss, accuracy, all_predictions, all_labels


print("Starting training...")
for epoch in range(num_epochs):
    print(f"\nEpoch {epoch + 1}/{num_epochs}")
    
 
    train_loss, train_acc = train_epoch(model, train_loader, optimizer, device)
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}")

    test_loss, test_acc, predictions, true_labels = evaluate_model(model, test_loader, device)
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")

print("\nTraining completed!")


print("\nFinal Results:")
print(f"Final Test Accuracy: {test_acc:.4f}")
print("\nClassification Report:")
print(classification_report(true_labels, predictions, target_names=['Class 0', 'Class 1']))

def predict_text(text, model, tokenizer, device, max_length=128):
    model.eval()
    
    encoding = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=max_length,
        return_tensors='pt'
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = F.softmax(outputs.logits, dim=-1)
        predicted_class = torch.argmax(predictions, dim=-1)
        confidence = torch.max(predictions).item()
    
    return predicted_class.item(), confidence


sample_text = "This is a sample text for prediction"
pred_class, confidence = predict_text(sample_text, model, tokenizer, device)
print(f"\nSample prediction:")
print(f"Text: '{sample_text}'")
print(f"Predicted class: {pred_class}, Confidence: {confidence:.4f}")


model.save_pretrained('./bert_model')
tokenizer.save_pretrained('./bert_model')
print("\nModel saved to './bert_model'")

Training samples: 25000
Test samples: 25000
Unique labels in train: {0, 1}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Using device: cuda
Training batches: 1563
Test batches: 1563
Starting training...

Epoch 1/3
Train Loss: 0.3291, Train Accuracy: 0.8545
Test Loss: 0.2683, Test Accuracy: 0.8858

Epoch 2/3
Train Loss: 0.1901, Train Accuracy: 0.9246
Test Loss: 0.2947, Test Accuracy: 0.8880

Epoch 3/3
Train Loss: 0.0945, Train Accuracy: 0.9662
Test Loss: 0.4012, Test Accuracy: 0.8799

Training completed!

Final Results:
Final Test Accuracy: 0.8799

Classification Report:
              precision    recall  f1-score   support

     Class 0       0.87      0.89      0.88     12500
     Class 1       0.89      0.87      0.88     12500

    accuracy                           0.88     25000
   macro avg       0.88      0.88      0.88     25000
weighted avg       0.88      0.88      0.88     25000


Sample prediction:
Text: 'This is a sample text for prediction'
Predicted class: 1, Confidence: 0.5349

Model saved to './bert_model'
